In [98]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## our CNN

In [99]:
import torch
import torch.nn as nn
import torch.optim as optim
import datasets as ds_own
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm
import numpy as np


In [100]:
class ConvolutionalNeuralNetwork():
    
    def train_step(self, data, optimizer, criterion):
        x, y = data

        optimizer.zero_grad()

        logits = self(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}

    
    def test_step(self, data, criterion):
        x, y = data

        logits = self(x)
        loss = criterion(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()

        return {'loss': loss, 'accuracy': accuracy}
    
    def Conv2d_output_size(self, w, k, s, p):
        '''
        w - width of input image
        k - kernel size
        s - stride
        p - padding
        '''
        return np.floor((w - k + 2 * p) / s + 1)

In [101]:
class CNN_3_class(nn.Module, ConvolutionalNeuralNetwork):
    def __init__(self, num_classes = 10
                ,kernel_size1=11
                ,kernel_size2=2
                ,stride=1
                ,padding=1
                ,number_of_filters0=32
                ,number_of_filters1=32
                ,length_of_input0=32
                ,no_neurons = 128
                ,dr=nn.Dropout(p=0)
                ,activation_function=torch.relu):
        super(CNN_3_class, self).__init__()
        self.conv1 = nn.Conv2d(3, number_of_filters0, kernel_size1, stride, padding)
        self.pool1 = nn.MaxPool2d(2)
        length0 = self.Conv2d_output_size(length_of_input0, kernel_size1, stride, padding)//2
        self.conv2 = nn.Conv2d(number_of_filters0, number_of_filters1, kernel_size2, stride, padding)
        self.pool2 = nn.MaxPool2d(2)
        length1 = self.Conv2d_output_size(length0, kernel_size2, stride, padding)//2
        self.fc1 = nn.Linear(int(length1*length1*number_of_filters1), no_neurons)
        self.fc2 = nn.Linear(no_neurons, num_classes)
        self.dr = dr
        self.activation_function = activation_function

    def forward(self, x):
        x = self.pool1(self.activation_function(self.conv1(x)))
        x = self.pool2(self.activation_function(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.activation_function(self.fc1(x))
        x = self.dr(x)
        x = self.fc2(x)
        return x


### Cifar10 datasets and loaders

In [102]:
train_dataset = ds_own.cifar_train
val_dataet = ds_own.cifar_val
test_loader = ds_own.val_loader
train_loader = ds_own.train_loader

cifar_basic_aug = ds_own.cifar_basic_aug
cifar_mixup = ds_own.cifar_mixup
cifar_cutout = ds_own.cifar_cutout

basic_aug_loader = ds_own.basic_aug_loader
mixup_loader = ds_own.mixup_loader
cutout_loader = ds_own.cutout_loader

In [103]:
learning_rate = 0.001
# batch_size = 64 TEGO NIE UZYWAMY A JESLI CHCEMY TO W LOADERZE TRZEBA DODAC
num_epochs = 2
optimizer = optim.Adam
activation_func = nn.ReLU()
# dropout_rate = 0.1
criterion = nn.CrossEntropyLoss()

# Create model, criterion, and optimizer
model = CNN_3_class(num_classes = 10
                    ,kernel_size1=7
                    ,kernel_size2=3
                    ,stride=1
                    ,padding=1
                    ,number_of_filters0=32
                    ,number_of_filters1=32
                    ,length_of_input0=32
                    ,no_neurons = 128
                    ,dr=nn.Dropout(p=0)
                    ,activation_function=torch.relu)
                    

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
epochs = num_epochs
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    train_losses = []
    train_accuracies = []
    for data in tqdm(train_loader):
        results = model.train_step(data, optimizer, criterion)
        train_losses.append(results['loss'].item())
        train_accuracies.append(results['accuracy'].item())

    # Calculate average training loss and accuracy for the epoch
    avg_train_loss = sum(train_losses) / len(train_losses)
    avg_train_accuracy = sum(train_accuracies) / len(train_accuracies)
    print(f'Train loss: {avg_train_loss:.4f}, Train accuracy: {avg_train_accuracy:.4f}')

    # Test the model
    val_losses = []
    val_accuracies = []
    with torch.no_grad():
        for data in tqdm(test_loader):
            results = model.test_step(data, criterion)
            val_losses.append(results['loss'].item())
            val_accuracies.append(results['accuracy'].item())

    # Calculate average test loss and accuracy for the epoch
    avg_validation_loss = sum(val_losses) / len(val_losses)
    avg_validation_accuracy = sum(val_accuracies) / len(val_accuracies)
    print(f'Validation loss: {avg_validation_loss:.4f}, Validation accuracy: {avg_validation_accuracy:.4f}')


Epoch 1/2


  5%|▌         | 68/1250 [00:02<00:47, 24.95it/s]


KeyboardInterrupt: 

## CNN from article about weighted random search

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import datasets
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from tqdm import tqdm

class MyCNN(nn.Module, ConvolutionalNeuralNetwork):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=736, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=736, out_channels=508, kernel_size=3, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=508, out_channels=664, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=664, out_channels=916, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv5 = nn.Conv2d(in_channels=916, out_channels=186, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=186, out_channels=352, kernel_size=3, padding=1)
        self.linear = nn.Linear(in_features=22528, out_features=1229)
        self.output = nn.Linear(in_features=1229, out_features=num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.maxpool2(x)
        x = self.conv5(x)
        x = nn.functional.relu(x)
        x = self.conv6(x)
        x = nn.functional.relu(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        x = nn.functional.relu(x)
        x = self.output(x)
        return x

## another aproach to alexnet

In [4]:
import torch.nn as nn
import torchvision.models as models

class PretrainedAlexNet(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super().__init__()

        self.model = models.alexnet(pretrained=pretrained)
        
        # Modify the last fully connected layer to output num_classes
        self.model.classifier[-1] = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x
    


In [5]:
from datasets import CifarDataset

In [7]:
# Path to training and validation directories
TRAIN_DIR = 'Cifar10\\train'
VAL_DIR = 'Cifar10\\val'

# Path to dataframe with labels for training and validation data
TRAIN_LABELS = 'Cifar10\\trainLabels.csv'
VAL_LABELS = 'Cifar10\\valLabels.csv'

# List of class names
CLASS_NAMES = ['frog', 'truck', 'deer', 'automobile', 'bird', 'horse', 'ship', 'cat', 'dog',
 'airplane']

# Dictionary for encoding class names
CLASS_DICT = {CLASS_NAMES[i]: i for i in range(len(CLASS_NAMES))}

# Size in pixels of single image
IMG_SIZE=32

# import transforms
import torchvision.transforms as transforms
transformer = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


# Create pyTorch Dataset instances of training and validation data
cifar_train = CifarDataset(root_dir = TRAIN_DIR, labels=TRAIN_LABELS, 
                           transform=transformer, class_dict=CLASS_DICT)
cifar_val = CifarDataset(root_dir = VAL_DIR, labels=VAL_LABELS, 
                         transform=transformer, class_dict=CLASS_DICT)

train_loader = DataLoader(cifar_train, batch_size=32, shuffle=True)
test_loader = DataLoader(cifar_val, batch_size=32, shuffle=False)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

# Prepare CIFAR10 dataset and dataloaders
transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# Initialize model and move to device
model = PretrainedAlexNet(num_classes=num_classes).to(device)


c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# print('1')
a = 0
# Train the model


from tqdm import tqdm

for epoch in range(num_epochs):
    running_loss = 0.0
    a = 0
    for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        inputs, labels = data[0], data[1]
        optimizer.zero_grad()
        try:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        except:
            a += 1
            continue
    print('Epoch {} loss: {:.3f}'.format(epoch+1, running_loss/len(train_loader)))
    print('Finished Training, {} errors'.format(a))



 52%|█████▏    | 651/1250 [07:19<07:05,  1.41it/s]

In [ ]:

# Evaluate the model
model.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()
    print('Test accuracy: {:.2f}%'.format(100 * total_correct / total_samples))


## VGG16

In [104]:
import torch.nn as nn
import torchvision.models as models

class PretrainedVGG16(nn.Module):
    def __init__(self, num_classes=10, pretrained=True):
        super().__init__()

        self.model = models.vgg16(pretrained=pretrained)
        
        # Modify the last fully connected layer to output num_classes
        self.model.classifier[-1] = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x


In [105]:
from datasets import CifarDataset

In [106]:
# Path to training and validation directories
TRAIN_DIR = 'Cifar10\\train'
VAL_DIR = 'Cifar10\\val'

# Path to dataframe with labels for training and validation data
TRAIN_LABELS = 'Cifar10\\trainLabels.csv'
VAL_LABELS = 'Cifar10\\valLabels.csv'

# List of class names
CLASS_NAMES = ['frog', 'truck', 'deer', 'automobile', 'bird', 'horse', 'ship', 'cat', 'dog',
 'airplane']

# Dictionary for encoding class names
CLASS_DICT = {CLASS_NAMES[i]: i for i in range(len(CLASS_NAMES))}

# Size in pixels of single image
IMG_SIZE=32

# import transforms
import torchvision.transforms as transforms
transformer = transforms.Compose([
    transforms.Resize(256),               # Resize the input image to 256x256
    transforms.CenterCrop(224),           # Crop the center 224x224 region of the image
    transforms.ToTensor(),                # Convert the image to a PyTorch tensor
    transforms.Normalize(                 # Normalize the image
        mean=[0.485, 0.456, 0.406],        #   using the standard ImageNet mean and std
        std=[0.229, 0.224, 0.225])
])

# Create pyTorch Dataset instances of training and validation data
cifar_train = CifarDataset(root_dir = TRAIN_DIR, labels=TRAIN_LABELS, 
                           transform=transformer, class_dict=CLASS_DICT)
cifar_val = CifarDataset(root_dir = VAL_DIR, labels=VAL_LABELS, 
                         transform=transformer, class_dict=CLASS_DICT)

train_loader = DataLoader(cifar_train, batch_size=32, shuffle=True)
test_loader = DataLoader(cifar_val, batch_size=32, shuffle=False)

In [107]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

# Initialize model and move to device
model = PretrainedVGG16(num_classes=num_classes).to(device)


c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\mikol/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:11<00:00, 47.5MB/s] 


In [108]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# print('1')
a = 0
# Train the model


from tqdm import tqdm

for epoch in range(num_epochs):
    running_loss = 0.0
    a = 0
    for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        inputs, labels = data[0], data[1]
        optimizer.zero_grad()
        try:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        except:
            a += 1
            continue
    print('Epoch {} loss: {:.3f}'.format(epoch+1, running_loss/len(train_loader)))
    print('Finished Training, {} errors'.format(a))



  4%|▍         | 53/1250 [13:45<3:19:25, 10.00s/it]  